In [143]:
import pandas as pd
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import os
from tqdm import tqdm
import sliding_window
from sklearn.neighbors import LocalOutlierFactor

import datetime, time
import importlib
importlib.reload(sliding_window)

<module 'sliding_window' from 'C:\\Users\\suhwanlee\\Desktop\\project\\streaming_anomaly_detect\\sliding_window.py'>

In [144]:
file_name = './data/loan_baseline.pnml_noise_0.15_iteration_1_seed_614_sample.csv'

dataset = stream.iter_csv(
            file_name
#             './data/loan_baseline.pnml_noise_0.15_iteration_1_seed_614_simple.csv',
            )

totallength = len(list(dataset))

In [145]:
dataset = stream.iter_csv(
            file_name,
            drop=['noise', 'lifecycle:transition', 'Variant', 'Variant index'],
            )
enctype = 'Index-base'

In [146]:
key_pair = {
'Case ID':'caseid',
'Activity':'activity',
# 'Resource':'resource',
'Complete Timestamp':'ts',
}
catatars= ['activity']#,'resource']

case_dict ={}
training_models ={}

casecount = 0
rowcounter = 0
resultdict ={}
acc_dict ={}
prefix_wise_window = {}
prediction_result = {}
graceperiod_finish=0
finishedcases = set()

In [147]:
# Sliding window for training setting
window_size = 50
retraining_size = 10
training_window = sliding_window.training_window(window_size,retraining_size)
contamination = 0.25

In [148]:
def display_progress(row_counting, total_length, interval=500):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(case_dict)))

In [149]:
def training_stage(window, training_models):
    '''
    Manage training stage of streaming anomaly detection
    ----------
    Parameters
    window: class training_window
        Sliding window with training data
    training_models: dict
        Trained detector by prefix stored in. Default is randomforest
    ----------
    Return
    training_models
    '''
    pw_window = window.prefix_wise_window()
    for x in pw_window:
        clf  = LocalOutlierFactor(contamination= contamination, novelty=True)
        try:
            clf.fit(pw_window[x][0])
            if 'detector_%s'%(x) not in training_models:
                training_models['detector_%s'%(x)] =[0,0]
            training_models['detector_%s'%(x)][0] += 1
            training_models['detector_%s'%(x)][1] = clf
        except:
            pass
    return training_models

In [150]:
def predict_activity_proba(last_event):
    '''
    Predict next activity prediction 
    
    Parameters
    ----------
    last_event: case_bin
    
    Return
    ----------
    modelid, prediction
    
    '''
    feature_matrix = prefix_wise_window['window_%s'%(last_event.prefix_length)][0].columns.values
    current_event = utils.readjustment_training(last_event.encoded, feature_matrix)
    current_event = pd.Series(current_event).to_frame().T
    prediction = [training_models['detector_window_%s'%(last_event.prefix_length)][1].predict_proba(current_event), training_models['detector_window_%s'%(last_event.prefix_length)][1].classes_]
    modelid = training_models['detector_window_%s'%(last_event.prefix_length)][0]

    return modelid, prediction

In [151]:
def first_event(case_bin):
    '''
    Generate start event before first event
    '''
    print(case_bin.event['ts'])
    empty_data ={'activity':'Start signal', 'ts':datetime.datetime.strftime(case_bin.event['ts'], '%Y-%m-%d %H:%M:%S')}
    start_event = prefix_bin(case_bin.caseid, empty_data)
    start_event.set_prefix_length(0)
    start_event.update_encoded(catattrs=catatars,enctype=enctype)
    start_event.update_truelabel(case_bin.event['activity'])
    return start_event

In [152]:
training_time =[]

In [153]:
start_time = time.time()

for x,y in dataset:
    display_progress(rowcounter, totallength)
    rowcounter +=1
    
    utils.dictkey_chg(x, key_pair)
    # Event stream change dictionary keys
    x['ts'] = x['ts'][:-4]
    
    # Check label possible
    
    # Initialize case by prefix length
    caseid = x['caseid']
    x.pop('caseid')
    
    case_bin = prefix_bin(caseid, x)
    
    if caseid not in list(case_dict.keys()):
        case_dict[caseid] = []
        case_bin.set_prefix_length(1)
        
    elif caseid in finishedcases:
        continue
    
    else:
        case_bin.set_prefix_length(len(case_dict[caseid])+1)
        case_bin.set_prev_enc(case_dict[caseid][-1])
    
    # Encode event and cases and add to DB
    ts = case_bin.event['ts']
    case_bin.update_encoded(catattrs=catatars,enctype=enctype)
    
    # Set current activity as outcome of previous event
    if case_bin.prefix_length != 1:
        case_bin.prev_enc.update_truelabel(x['activity'])

    # First prediction for current event
    
    last_event = case_bin
    modelid = 'None'
    prediction = 'Not Available'

    if len(training_window.getAllitems()) !=0:
        if 'window_%s'%(last_event.prefix_length) in list(prefix_wise_window.keys()) and 'detector_window_%s'%(last_event.prefix_length) in training_models.keys():
#             modelid, prediction = predict_activity_proba(last_event)
            feature_matrix = prefix_wise_window['window_%s'%(last_event.prefix_length)][0].columns.values
            current_event = utils.readjustment_training(last_event.encoded, feature_matrix)
            current_event = pd.Series(current_event).to_frame().T
            try:
                prediction = [training_models['detector_window_%s'%(last_event.prefix_length)][1].predict(current_event)]
            except:
                prediction = 'Not Available'
            modelid = training_models['detector_window_%s'%(last_event.prefix_length)][0]
    case_bin.update_prediction((modelid, (prediction,ts)))        
            
    # Update training window and finish the case
    if x['activity'] == 'End':
        training_window.update_window({caseid: case_dict[caseid]})        
        if training_window.retraining == training_window.retraining_count:            
            train_start = time.time()
            training_models = training_stage(training_window, training_models)
            train_end = time.time()
            training_time.append(train_end-train_start)
            prefix_wise_window = training_window.prefix_wise_window()
            
        resultdict[caseid] = case_dict[caseid]
        case_dict.pop(caseid)

        casecount +=1
        for x in case_dict:
            last_event = case_dict[x][-1]
            modelid = 'None'
            prediction = 'Not Available'

            if len(training_window.getAllitems()) !=0:
                prefix_wise_window = training_window.prefix_wise_window()
                if 'window_%s'%(last_event.prefix_length) in list(prefix_wise_window.keys()) and 'detector_window_%s'%(last_event.prefix_length) in training_models.keys():
#                     modelid, prediction = predict_activity_proba(last_event)

                    feature_matrix = prefix_wise_window['window_%s'%(last_event.prefix_length)][0].columns.values
                    current_event = utils.readjustment_training(last_event.encoded, feature_matrix)
                    current_event = pd.Series(current_event).to_frame().T
                    try:
                        prediction = [training_models['detector_window_%s'%(last_event.prefix_length)][1].predict(current_event)]
                    except:
                        prediction = 'Not Available'
                    modelid = training_models['detector_window_%s'%(last_event.prefix_length)][0]
            case_dict[x][-1].update_prediction((modelid, (prediction,ts)))        
        training_window.reset_retraining_count()
    else:
        case_dict[caseid].append(case_bin)


end_time = time.time()

0.0 % Case finished: 0 Running cases: 0


c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (10). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (10). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (10). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (10). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is gre

5.8 % Case finished: 29 Running cases: 1


c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (19). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (18). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (13). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (11). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is gre

c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (19). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (11). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is gre

...

c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (15). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (11). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is gre

92.87 % Case finished: 490 Running cases: 1


c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (13). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (13). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (11). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is gre

98.68 % Case finished: 518 Running cases: 1


c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (18). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (18). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "
c:\python\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (20) is greater than the total number of samples (11). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is gre

In [136]:
print(training_models)

{'detector_window_1': [52, LocalOutlierFactor(novelty=True)], 'detector_window_2': [52, LocalOutlierFactor(novelty=True)], 'detector_window_3': [52, LocalOutlierFactor(novelty=True)], 'detector_window_4': [52, LocalOutlierFactor(novelty=True)], 'detector_window_5': [52, LocalOutlierFactor(novelty=True)], 'detector_window_6': [52, LocalOutlierFactor(novelty=True)], 'detector_window_7': [52, LocalOutlierFactor(novelty=True)], 'detector_window_8': [52, LocalOutlierFactor(novelty=True)], 'detector_window_9': [52, LocalOutlierFactor(novelty=True)], 'detector_window_10': [52, LocalOutlierFactor(novelty=True)], 'detector_window_11': [52, LocalOutlierFactor(novelty=True)], 'detector_window_12': [52, LocalOutlierFactor(novelty=True)], 'detector_window_13': [52, LocalOutlierFactor(novelty=True)], 'detector_window_14': [52, LocalOutlierFactor(novelty=True)], 'detector_window_15': [52, LocalOutlierFactor(novelty=True)], 'detector_window_16': [52, LocalOutlierFactor(novelty=True)], 'detector_window

In [137]:
print(np.mean(training_time), np.std(training_time))

0.13972373192126936 0.025847422737158014


In [138]:
print((end_time-start_time)/60)

0.44363274971644084


In [139]:
original_df = pd.read_csv(file_name)


In [142]:
for_confusion_matrix = {}

global_true =[]
global_pred = []
counting_normal = 0
for caseid in list(resultdict.keys()):

    for_confusion_matrix[int(caseid)] =[]
    
    prediction_list = []
    
    df = original_df[original_df['Case ID'] == int(caseid)].reset_index(drop=True)
    for pos, t in enumerate(resultdict['%s'%(caseid)]):
        
        predictions = list(t.predicted.values())[0][0]    
        if predictions  == 'Not Available':
            predictions_label = 'Not Available'
        else:
            predictions_label = predictions[0][0]

        if predictions_label == 1:
            predictions_label = 'Normal'
        elif predictions_label == -1:
            predictions_label = 'Anomalous'

        if t.event['activity'] != 'Start':
            prediction_list.append(predictions_label)
            
    true_label_list = []

    labellist = list(df['noise'])
    actlist = list(df['Activity'])
    for pos, t in enumerate(labellist):
        if t == 'Start' or t == 'End':
            continue
        elif t == 'true':
            true_label = 'Anomalous'
        else:
            true_label = 'Normal'
        true_label_list.append(true_label)

    
    for pos, p in enumerate(prediction_list):
        global_pred.append(p)
        global_true.append(true_label_list[pos])
saving_data = {'y_true':global_true, 'y_pred':global_pred}

import pickle
saving_file_name = file_name.split('/')[-1][:-4]
with open('./result/lof_cont%s_window%s_%s.pkl'%(contamination, window_size, saving_file_name), 'wb') as fp:
    pickle.dump(saving_data, fp)
#     print(caseid, len(true_label_list), len(prediction_list))

#     for t in true_label_list:
#         global_true.append(t)
#     print(prediction_list)
#     print(true_label_list)

['Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Availabl